# Prevendo saídas do banco

Este projeto objetivou cosntruir um modelo que preveja se um cliente cessará sua conta num banco ou não, ou seja, "sair do banco". Para isso, o projeto ajusta e teste diferentes modelos de aprendizado de máquina utlizando dados do *fictício* Beta Bank. Os dados são a cartela de clientes com seus comportamentos e a informção sobre rescisões de contratos com o banco.

## Importando bibliotecas e dados

In [1]:
### importando bibliotecas
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

In [2]:
### importando dados
clients = pd.read_csv('/datasets/Churn.csv')

## Processando dados

### Estudando os dados

In [3]:
### visualizando algumas linhas
clients.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


**Comentário**: As variáveis de linha (`RowNumber`), identificação (`CustomerId`) e sobrenome (`Surname`) do cliente não são características úteis para a predição, são apenas identificadores. A variável de geografia (`Geography`) precisará ser transformada em categorias não-ordinais, assim como a variável de gênero (`Gender`).

In [4]:
### verificando tipo das variáveis
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


**Comentário**: A variável de identificação dos clientes (`CustomerId`) poderia conter dados do tipo texto. A variável de tempo de serviço (`Tenure`) possui dados faltantes.

In [5]:
### possui entradas duplicadas?
print("Número de entradas duplicadas:", sum(clients.duplicated()) )

Número de entradas duplicadas: 0


In [6]:
### descrevendo distribuição dos dados
clients.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


**Comentário**: As médias em geral estão próximas da mediana, indicando uma distribuição simétrica dos dados em geral. A média será o descritor de tendência central. 

### Tratando dados

In [7]:
### transformando gender em dummy
clients["Male"] = pd.get_dummies(clients["Gender"], drop_first= True)

In [8]:
### transformando geography em dummies
clients[["France", "Germany", "Spain"]] = pd.get_dummies(clients["Geography"], drop_first= False)

In [9]:
### retirando variáveis categóricas originais
clients = clients.drop(["Gender", "Geography"], axis = 1)

In [10]:
### substituindo dados faltantes de Tenure pela média total da variável
clients["Tenure"] = clients["Tenure"].fillna(value= clients["Tenure"].mean() )

In [11]:
clients.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Male,France,Germany,Spain
0,1,15634602,Hargrave,619,42,2.0,0.00,1,1,1,101348.88,1,0,1,0,0
1,2,15647311,Hill,608,41,1.0,83807.86,1,0,1,112542.58,0,0,0,0,1
2,3,15619304,Onio,502,42,8.0,159660.80,3,1,0,113931.57,1,0,1,0,0
3,4,15701354,Boni,699,39,1.0,0.00,2,0,0,93826.63,0,0,1,0,0
4,5,15737888,Mitchell,850,43,2.0,125510.82,1,1,1,79084.10,0,0,0,0,1


**Comentário**: Transformei a variável categórica `Gender` na variável dummy `Male`. Também transformei a variável `Geography` nas dummies `France`, `Germany`, `Spain`. Neste último caso, as dummies são redundantes, mas descartar alguma delas implicaria em perda de informação. Por último, substitui os valores faltantes de `Tenure` pela média da variável.

## Objetivo & Caraterísticas 

### Separando objetivos e características

In [12]:
target = clients['Exited']

In [13]:
features = clients.drop(["RowNumber", "CustomerId", "Surname", "Exited"] , axis=1)

In [14]:
features.head(5)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Male,France,Germany,Spain
0,619,42,2.0,0.00,1,1,1,101348.88,0,1,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,0,1
2,502,42,8.0,159660.80,3,1,0,113931.57,0,1,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,1,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,0,1


### Verificando equílibrio de classes 

In [15]:
print("Número de clientes que saíram:", sum(target == 1))
print("Número de clientes que permancem:", sum(target == 0))

Número de clientes que saíram: 2037
Número de clientes que permancem: 7963


In [16]:
print("Porcentagem de clientes que saíram:", round(100* (sum(target == 1) / len(target)), 2),"%")

Porcentagem de clientes que saíram: 20.37 %


**Comentário**: No objetivo, a classe "saiu" (1) é consideravelmente menos frequente. Logo, serão aplicadas estratégias de equilíbrio de classe.

### Estratégias de equilíbrio de classe

In [17]:
### defindindo função para aumentar a classe rara
def upsample(features, target):
    ## separando por classe
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    ## verificando proporção de classes
    prop = round(len(target_zeros) / len(target_ones))
    repeat = prop
    ## usando proporção para equilibrar
    if prop > 1 :
        features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
        target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    elif prop < 1:
        features_upsampled = pd.concat(([features_zeros] * repeat)+ [features_ones])
        target_upsampled = pd.concat(([target_zeros] * repeat) + [target_ones])
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345 )
    ## retornando características e objetivos equilibrados
    return features_upsampled, target_upsampled

In [18]:
### equilibrando classes pra cima
features_up, target_up = upsample(features, target)

In [19]:
print("Porcentagem de clientes que saíram (amostra up):", round(100* (sum(target_up == 1) / len(target_up)), 2),"%")

Porcentagem de clientes que saíram (amostra up): 50.57 %


In [20]:
### defindindo função para diminuir a classe frequente
def downsample(features, target):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    ## verificando proporção de classes
    prop = round(len(target_zeros) / len(target_ones))
    if prop > 1:
        fraction = 1/prop
        features_downsampled = pd.concat([features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
        target_downsampled = pd.concat([target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    elif prop < 1:
        fraction = prop
        features_downsampled = pd.concat([features_ones.sample(frac=fraction, random_state=12345)] + [features_zeros])
        target_downsampled = pd.concat([target_ones.sample(frac=fraction, random_state=12345)] + [target_zeros])
    features_downsampled, target_downsampled = shuffle(features_downsampled, target_downsampled, random_state=12345)
    ## retornando características e objetivos equilibrados
    return features_downsampled, target_downsampled

In [21]:
### equilibrando classes para baixo
features_dw, target_dw = downsample(features, target)

In [22]:
print("Porcentagem de clientes que saíram (amostra down):", round(100* (sum(target_dw == 1) / len(target_dw)), 2),"%")

Porcentagem de clientes que saíram (amostra down): 50.57 %


**Comentário**: Apliquei duas técnicas para equilibrar as classes do objetivo:

1) Superamostragem da classe rara ("1") = 'equilibrado para cima"

2) Subamostragem da classe frequente ("0") = 'equilibrado para baixo"

## Dividindo dados de treino, validação e teste

In [23]:
### definindo função para dividir treino, validação e teste
def divide_data(features, target): 
    features_tr, features_va_te, target_tr, target_va_te = train_test_split(features, target, train_size=0.6, random_state= 42)
    features_va, features_te, target_va, target_te = train_test_split(features_va_te, target_va_te, train_size=0.5, random_state=42)
    return features_tr, features_va, features_te, target_tr, target_va, target_te

In [24]:
### dividindo dados não-equilibrados
features_tr, features_va, features_te, target_tr, target_va, target_te = divide_data(features, target)

In [25]:
### dividindo dados equilibrados para cima
features_tr_up, features_va_up, features_te_up, target_tr_up, target_va_up, target_te_up = divide_data(features_up, target_up)

In [26]:
### dividindo dados equilibrados para baixo
features_tr_dw, features_va_dw, features_te_dw, target_tr_dw, target_va_dw, target_te_dw = divide_data(features_dw, target_dw)

**Comentário**: Dididi todos as três versões dos dados em 60% treino, 20% validação e 20% teste.

## Ajustando modelo `Logistic Regression`

In [27]:
### definindo função para ajuste de Losgistic Regression
def set_logistic_model(features_train, target_train, features_valid, target_valid):
    ## ajustando
    logis_model = LogisticRegression(solver= 'lbfgs', max_iter=1000, random_state=42)
    logis_model.fit(features_train, target_train)
    ## predições
    prediction_valid = logis_model.predict(features_valid)
    probs_valid = logis_model.predict_proba(features_valid)
    probs_one_valid = probs_valid[:, 1]
    ## avaliando modelos
    logis_f1 = f1_score(target_valid, prediction_valid)
    logis_auc = roc_auc_score(target_valid, probs_one_valid)
    ## resultados
    print(f'Modelo logístico com F1: {round(logis_f1,2)}, e AUC: {round(logis_auc,2)}')
    return logis_model

In [28]:
### ajustando modelo aos dados não-equilibrados
logis_model_non = set_logistic_model(features_train = features_tr, target_train = target_tr, features_valid = features_va, target_valid = target_va)

Modelo logístico com F1: 0.11, e AUC: 0.65


In [29]:
### ajustando os dados equilibrados para cima
logis_model_up = set_logistic_model(features_train = features_tr_up, target_train = target_tr_up, features_valid = features_va_up, target_valid = target_va_up)

Modelo logístico com F1: 0.62, e AUC: 0.59


In [30]:
### ajustando os dados equilibrados para baixo
logis_model_dw = set_logistic_model(features_train = features_tr_dw, target_train = target_tr_dw, features_valid = features_va_dw, target_valid = target_va_dw)

Modelo logístico com F1: 0.67, e AUC: 0.7


**Comentário**: O modelo `Logistic Regression` ajustado aos dados equilibrados para baixo teve a melhor performance, seguido do modelo para os dados equilibrados para cima, e não-equilibrados.

## Ajustando modelo `Decicion Tree`

In [31]:
### definindo função para ajuste de Decision Tree
def set_tree_model(features_train, target_train, features_valid, target_valid):
    best_tree_model = None
    best_depth = 0
    best_tree_f1 = 0
    best_tree_auc = 0
    for depth in range(1,30):
        ## ajustando
        tree_model = DecisionTreeClassifier(max_depth= depth, random_state = 42)
        tree_model.fit(features_train, target_train)
        ## predizendo
        prediction_valid = tree_model.predict(features_valid)
        probs_valid = tree_model.predict_proba(features_valid)
        probs_one_valid = probs_valid[:, 1]
        ## avaliando
        tree_f1 = f1_score(target_valid, prediction_valid)
        tree_auc = roc_auc_score(target_valid, probs_one_valid)
        if (tree_f1 > best_tree_f1):
            best_tree_model = tree_model
            best_depth = depth
            best_tree_f1 = tree_f1
            best_tree_auc = tree_auc
    print(f'Melhor árvore com profundidade: {best_depth}, F1: {round(best_tree_f1,2)}, AUC: {round(best_tree_auc,2)}')
    return best_tree_model

In [32]:
### ajustando o modelo aos dados não-equilibrados
tree_model_non = set_tree_model(features_train = features_tr, target_train = target_tr, features_valid = features_va, target_valid = target_va)

Melhor árvore com profundidade: 6, F1: 0.52, AUC: 0.8


In [33]:
### ajustando o modelo aos dados equilibrados para cima
tree_model_up = set_tree_model(features_train = features_tr_up, target_train = target_tr_up, features_valid = features_va_up, target_valid = target_va_up)

Melhor árvore com profundidade: 24, F1: 0.91, AUC: 0.9


In [34]:
### ajustando o modelo aos dados equilibrados para baixo
tree_model_dw = set_tree_model(features_train = features_tr_dw, target_train = target_tr_dw, features_valid = features_va_dw, target_valid = target_va_dw)

Melhor árvore com profundidade: 6, F1: 0.78, AUC: 0.84


**Comentário**: O modelo `Decision Tree` ajustado aos dados equilibrados para cima teve a melhor performance, seguido do modelo para os dados equilibrados para baixo, e não-equilibrados.

## Ajustando modelo `Random Forest`

In [35]:
### definindo função para ajuste de Random Forest
def set_forest_model(features_train, target_train, features_valid, target_valid):
    best_forest_model = None
    best_est = 0
    best_depth = 0
    best_forest_f1 = 0
    best_forest_auc = 0
    for est in range(1, 31):
        for depth in range(1,21):
            forest_model = RandomForestClassifier(n_estimators= est, max_depth= depth, random_state = 42)
            forest_model.fit(features_train, target_train)
            ## predizendo
            prediction_valid = forest_model.predict(features_valid)
            probs_valid = forest_model.predict_proba(features_valid)
            probs_one_valid = probs_valid[:, 1]
            ## avaliando
            forest_f1 = f1_score(target_valid, prediction_valid)
            forest_auc = roc_auc_score(target_valid, probs_one_valid)
            if (forest_f1 > best_forest_f1):
                best_forest_model = forest_model
                best_est = est
                best_depth = depth
                best_forest_f1 = forest_f1
                best_forest_auc = forest_auc
    print(f'Melhor floresta com {best_est} árvores, de profundidade: {best_depth}, com F1: {round(forest_f1,2)}, AUC: {round(best_forest_auc,2)}')
    return best_forest_model

In [36]:
### ajustando aos dados não-equilibrados
forest_model_non = set_forest_model(features_train = features_tr, target_train = target_tr, features_valid = features_va, target_valid = target_va)

Melhor floresta com 29 árvores, de profundidade: 19, com F1: 0.54, AUC: 0.82


In [37]:
### ajsutando aos dados equilibrados para cima
forest_model_up = set_forest_model(features_train = features_tr_up, target_train = target_tr_up, features_valid = features_va_up, target_valid = target_va_up)

Melhor floresta com 26 árvores, de profundidade: 19, com F1: 0.93, AUC: 0.98


In [38]:
### ajustando aos dados equilibrados para baixo
forest_model_dw = set_forest_model(features_train = features_tr_dw, target_train = target_tr_dw, features_valid = features_va_dw, target_valid = target_va_dw)

Melhor floresta com 10 árvores, de profundidade: 9, com F1: 0.76, AUC: 0.85


**Comentário**: O modelo `Random tree` ajustado aos dados equilibrados para cima teve a melhor performance, seguido do modelo para os dados equilibrados para cima, e não-equilibrados.

**COMENTÁRIO GERAL**: Os modelos foram ajustados para maximizar o valor de F1, mas também tiveram o valor de AUC calculado. Contudo, os valores de F1 e AUC não estão claramente correlacionados. Em outras palavras, valores de AUC parecidos podem ocorrer em modelos com valores de F1 discrepantes. Além disso, o valor de AUC é mais 'otimistas' quanto o poder de predição dos modelos.

## Testando os modelos

In [39]:
### definindo função para testar os modelos 
def test_model(model, features_test, target_test):
    ## predizendo
    prediction_test = model.predict(features_test)
    probs_test = model.predict_proba(features_test)
    probs_one_test = probs_test[:, 1]
    ## avaliando
    f1 = f1_score(target_test, prediction_test)
    auc = roc_auc_score(target_test, probs_one_test)
    return f1, auc

In [40]:
### listando todos os modelos ajustados
model_list = [logis_model_non, logis_model_up, logis_model_dw, tree_model_non, tree_model_up, tree_model_dw, forest_model_non, forest_model_up, forest_model_dw]
model_names = ["logis_model_non", "logis_model_up", "logis_model_dw", "tree_model_non", "tree_model_up", "tree_model_dw", "forest_model_non", "forest_model_up", "forest_model_dw"]

In [41]:
### testando os modelos listados
f1_values= []
auc_values = []
for one_model in model_list:
    f1, auc = test_model(model = one_model, features_test = features_te, target_test = target_te)
    f1_values.append(f1)
    auc_values.append(auc)

In [42]:
### reunindo resultados
model_test_df = pd.DataFrame({"model": model_names, "F1": f1_values, "AUC": auc_values})
model_test_df

,model,F1,AUC
0,logis_model_non,0.166667,0.678734
1,logis_model_up,0.369062,0.579434
2,logis_model_dw,0.457689,0.721041
3,tree_model_non,0.586895,0.841004
4,tree_model_up,0.884058,0.961798
5,tree_model_dw,0.574468,0.847109
6,forest_model_non,0.593343,0.847792
7,forest_model_up,0.931447,0.996677
8,forest_model_dw,0.646604,0.896066


**Comentário**: O modelo `forest_model_up` obteve a melhor performance preditiva, com F1 de 0.93 e AUC de 0.99. De modo geral, os modelos ajustados a dados equilibrados tiveram melhor performance que os modelos ajustados a dados não-equilibrados. A métrica AUC 

## Conclusão

Este projeto teve o objetivo de cosntruir um modelo que preveja se um cliente cessará sua conta num banco ou não, ou seja, "sair do banco". Para isso, foram ajustados e testados diferents modelos de aprendizado de máquina. O modelo que melhor previu a saída dos clientes foi `forest_model_up`, que é baseado num algoritmo de `Random Forest`.

Todavia, a performance do melhor modelo dependeu de técnicas de equilíbrio de classes, visto que clientes que saem do banco são relativamente mais raros. Além disso, a escolha do melhor modelo foi basead na métrica F1, aparentemente mais conservativa do que a métrica AUC. 